## 4.分组

In [105]:
import pandas as pd
import numpy as np

### 4.1分组模式及对象

df.groupby(分组依据)[数据来源].使用操作,使用操作——如分别计算各组的平均值

In [106]:
df = pd.read_csv('data/learn_pandas.csv')
df.head()

,School,Grade,Name,Gender,Height,Weight,Transfer,Test_Number,Test_Date,Time_Record
0,Shanghai Jiao Tong University,Freshman,Gaopeng Yang,Female,158.9,46.0,N,1,2019/10/5,0:04:34
1,Peking University,Freshman,Changqiang You,Male,166.5,70.0,N,1,2019/9/4,0:04:20
2,Shanghai Jiao Tong University,Senior,Mei Sun,Male,188.9,89.0,N,2,2019/9/12,0:05:22
3,Fudan University,Sophomore,Xiaojuan Sun,Female,NaN,41.0,N,2,2020/1/3,0:04:08
4,Fudan University,Sophomore,Gaojuan You,Male,174.0,74.0,N,2,2019/11/6,0:05:22


#### 分组的依据和本质

In [107]:
# 按性别统计身高平均值
df.groupby('Gender')['Height'].mean()

Gender
Female    159.19697
Male      173.62549
Name: Height, dtype: float64

In [108]:
# 多组依据
df.groupby(['School', 'Gender'])['Height'].max()

# 条件分组,根据是否超平均值分组
print(df.Height.mean())
condition1 = df.Height > df.Height.mean()
df.groupby([condition1])['Height'].mean()

163.21803278688526


Height
False    157.428302
True     171.188312
Name: Height, dtype: float64

In [109]:
# 按照df的行数选择
item = np.random.choice(list('abc'), df.shape[0])

# 本质是按照条件中元素的值进行分组，如Ture一组，False一组
# 此处是a一组，b一组，c一组
df.groupby(item)['Height'].mean()

# 传入多个分组依据,则对其进行唯一组合
df.groupby([condition1, item])['Height'].mean()

Height   
False   a    157.058537
        b    158.609375
        c    156.742424
True    a    171.616000
        b    169.604762
        c    171.916129
Name: Height, dtype: float64

In [110]:
df['School'].drop_duplicates()

0    Shanghai Jiao Tong University
1                Peking University
3                 Fudan University
5              Tsinghua University
Name: School, dtype: object

In [111]:
# 分组的数组来源本质上是对应列的unique值
df.groupby(df['School'])['Height'].mean()
df.groupby([df['School'], df['Gender']])['Height'].mean()

School                         Gender
Fudan University               Female    158.776923
                               Male      174.212500
Peking University              Female    158.666667
                               Male      172.030000
Shanghai Jiao Tong University  Female    159.122500
                               Male      176.760000
Tsinghua University            Female    159.753333
                               Male      171.638889
Name: Height, dtype: float64

In [112]:
df.groupby(['School', 'Gender'])['Height'].mean()

School                         Gender
Fudan University               Female    158.776923
                               Male      174.212500
Peking University              Female    158.666667
                               Male      172.030000
Shanghai Jiao Tong University  Female    159.122500
                               Male      176.760000
Tsinghua University            Female    159.753333
                               Male      171.638889
Name: Height, dtype: float64

#### Groupby对象常见的属性

分组操作时，所有调用的方法都来自pandas中的groupby对象

In [113]:
gb = df.groupby(['School', 'Gender'])

# 分组个数4*2=8
gb.ngroups

8

In [114]:
res_map = gb.groups
res_map.keys()

# 返回字典，keys:分组组名，values:组中元素对应的列表索引
print(res_map)

{('Fudan University', 'Female'): [3, 15, 26, 28, 37, 39, 46, 49, 52, 63, 68, 70, 77, 84, 90, 105, 107, 108, 112, 129, 138, 144, 145, 157, 170, 173, 186, 187, 189, 195], ('Fudan University', 'Male'): [4, 41, 48, 66, 73, 82, 98, 131, 135, 152], ('Peking University', 'Female'): [9, 20, 29, 30, 32, 45, 57, 59, 75, 83, 86, 88, 96, 101, 120, 130, 132, 140, 159, 183, 185, 194], ('Peking University', 'Male'): [1, 35, 36, 38, 54, 61, 72, 99, 102, 116, 127, 147], ('Shanghai Jiao Tong University', 'Female'): [0, 6, 12, 13, 19, 22, 31, 42, 56, 58, 64, 65, 79, 85, 87, 89, 93, 103, 104, 109, 114, 115, 119, 121, 122, 123, 124, 141, 143, 148, 149, 155, 156, 161, 164, 166, 167, 172, 174, 188, 197], ('Shanghai Jiao Tong University', 'Male'): [2, 10, 21, 23, 50, 60, 71, 117, 134, 153, 165, 171, 184, 190, 192, 198], ('Tsinghua University', 'Female'): [5, 7, 8, 11, 14, 25, 27, 33, 34, 43, 44, 47, 51, 53, 55, 62, 67, 69, 78, 80, 81, 92, 97, 100, 106, 110, 111, 113, 118, 125, 126, 128, 133, 136, 137, 139, 14

In [115]:
# size方法,返回每组中元素的个数
gp_count = gb.size()
gp_count

School                         Gender
Fudan University               Female    30
                               Male      10
Peking University              Female    22
                               Male      12
Shanghai Jiao Tong University  Female    41
                               Male      16
Tsinghua University            Female    48
                               Male      21
dtype: int64

In [116]:
# 在相应的组中进行索引,组名Peking University,Female
res = gb.groups

# res.keys()是视图对象，要选取特定值需转换为列表，但是列表占用的内存会增大
keys = list(res.keys())
gb.get_group(keys[2]).iloc[:3, :3]

# 等价于
gb.get_group(('Peking University', 'Female')).iloc[:3, :3]

,School,Grade,Name
9,Peking University,Junior,Juan Xu
20,Peking University,Junior,Changjuan You
29,Peking University,Sophomore,Changmei Xu


### 4.2 聚合函数

#### 内置聚合函数

In [117]:
# 优先采用groupby对象的聚合函数，速度更快
df = pd.read_csv('data/learn_pandas.csv')
gp = df.groupby('Gender')[['Height', 'Weight']]
print(type(gp))
print(gp.max())
print(gp.mean())
gp.quantile(0.95)   # 0.95分位数

<class 'pandas.core.groupby.generic.DataFrameGroupBy'>
        Height  Weight
Gender                
Female   170.2    63.0
Male     193.9    89.0
           Height     Weight
Gender                      
Female  159.19697  47.918519
Male    173.62549  72.759259


,Height,Weight
Gender,,
Female,166.8,56.0
Male,185.9,84.4


#### agg聚合函数

In [118]:
gp.describe()   # 统计信息汇总

Height                                                              \
        count       mean       std    min      25%    50%      75%    max   
Gender                                                                      
Female  132.0  159.19697  5.053982  145.4  155.675  159.6  162.825  170.2   
Male     51.0  173.62549  7.048485  155.7  168.900  173.4  177.150  193.9   

       Weight                                                      
        count       mean       std   min   25%   50%    75%   max  
Gender                                                             
Female  135.0  47.918519  5.405983  34.0  44.0  48.0  52.00  63.0  
Male     54.0  72.759259  7.772557  51.0  69.0  73.0  78.75  89.0

In [119]:
# 1.同时使用多个函数
gp.agg(['max','mean', 'min'])

Height                   Weight                 
          max       mean    min    max       mean   min
Gender                                                 
Female  170.2  159.19697  145.4   63.0  47.918519  34.0
Male    193.9  173.62549  155.7   89.0  72.759259  51.0

In [120]:
# 2.对特定列使用特定的聚合函数
gp.agg({'Height':['max', 'mean', 'min'], 'Weight':'max'})

Height                   Weight
          max       mean    min    max
Gender                                
Female  170.2  159.19697  145.4   63.0
Male    193.9  173.62549  155.7   89.0

In [121]:
# 3.使用自定义函数,传入函数的参数为-列
gp.agg(lambda x: x.max()-x.min())

,Height,Weight
Gender,,
Female,24.8,29.0
Male,38.2,38.0


In [122]:
# gp通过df.groupby()生成
# 更改函数名称,sum变为change_sum
gp.agg([('极差', lambda x: x.max()-x.min()), ('change_sum', 'sum')])

Height            Weight           
           极差 change_sum     极差 change_sum
Gender                                    
Female   24.8    21014.0   29.0     6469.0
Male     38.2     8854.9   38.0     3929.0

In [123]:
gp.agg({'Height':[('平均值', 'mean')], 'Weight':[('最大值', 'max')]})

,Height,Weight
,平均值,最大值
Gender,,
Female,159.19697,63.0
Male,173.62549,89.0


### 4.3变换和过滤

返回值为同长度的序列

常见的内置变换函数为：cumcount/cumsum/cumprod/cummax/cummin

#### 内置变换函数和transform自定义变换函数

In [124]:
gp.cummax().head()

,Height,Weight
0,158.9,46.0
1,166.5,70.0
2,188.9,89.0
3,NaN,46.0
4,188.9,89.0


In [125]:
# 降序排列
# 在每个分组内进行排名
print(gp.size())

# 降序排列,按照gender分为female和male，其中female组144条记录，male组59条记录
# 平均排名,即两个相等的值2,3，平均排名后为2.5
res = gp.rank(method='average', ascending=True)

Gender
Female    141
Male       59
dtype: int64


In [129]:
# 自定义变换函数,对身高和体重标准化
gp.transform(lambda x: (x-x.mean())/x.std()).head()

# 接受传入标量的函数，并通过广播机制扩增
gp.transform('mean').head()

,Height,Weight
0,159.19697,47.918519
1,173.62549,72.759259
2,173.62549,72.759259
3,159.19697,47.918519
4,173.62549,72.759259


#### 数组的索引与过滤